# Data Wrangling Exercises

## Acquire (acquire.py)

### Zillow

For the following, iterate through the steps you would take to create functions: Write the code to do the following in a jupyter notebook, test it, convert to functions, then create the file to house those functions.

You will have a zillow.ipynb file and a helper file for each section in the pipeline.

**acquire & summarize**

1. Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
Only include properties with a transaction in 2017, and include only the last transaction for each properity (so no duplicate property ID's), along with zestimate error and date of transaction.
Only include properties that include a latitude and longitude value.



In [1]:
import pandas as pd
import acquire

In [2]:
df = acquire.get_zillow_cluster_data()

/usr/local/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (66) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


**acquire & summarize**
2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [3]:
df.shape

(77381, 70)

In [4]:
df.head()

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,...,logerror,pid,tdate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,14297519,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1727539,NaN,...,0.025595,14297519,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,17052889,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1387261,NaN,...,0.055619,17052889,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,14186244,NaN,NaN,261.0,NaN,NaN,NaN,NaN,11677,NaN,...,0.005383,14186244,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,12177905,NaN,NaN,261.0,2.0,NaN,NaN,NaN,2288172,NaN,...,-0.103410,12177905,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,10887214,NaN,NaN,266.0,2.0,NaN,NaN,1.0,1970746,NaN,...,0.006940,10887214,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN


In [5]:
df.dtypes

parcelid                    int64
typeconstructiontypeid    float64
storytypeid               float64
propertylandusetypeid     float64
heatingorsystemtypeid     float64
                           ...   
buildingclassdesc          object
heatingorsystemdesc        object
propertylandusedesc        object
storydesc                  object
typeconstructiondesc       object
Length: 70, dtype: object

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77381 entries, 0 to 77380
Data columns (total 70 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      77381 non-null  int64  
 1   typeconstructiontypeid        222 non-null    float64
 2   storytypeid                   50 non-null     float64
 3   propertylandusetypeid         77381 non-null  float64
 4   heatingorsystemtypeid         49440 non-null  float64
 5   buildingclasstypeid           15 non-null     float64
 6   architecturalstyletypeid      206 non-null    float64
 7   airconditioningtypeid         24953 non-null  float64
 8   id                            77381 non-null  int64  
 9   basementsqft                  50 non-null     float64
 10  bathroomcnt                   77381 non-null  float64
 11  bedroomcnt                    77381 non-null  float64
 12  buildingqualitytypeid         49672 non-null  float64
 13  c

In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
parcelid,77381.0,1.300715e+07,3.481346e+06,1.071186e+07,1.153830e+07,1.253157e+07,1.421183e+07,1.676893e+08
typeconstructiontypeid,222.0,6.040541e+00,5.572847e-01,4.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,1.300000e+01
storytypeid,50.0,7.000000e+00,0.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00
propertylandusetypeid,77381.0,2.618263e+02,5.141199e+00,3.100000e+01,2.610000e+02,2.610000e+02,2.660000e+02,2.750000e+02
heatingorsystemtypeid,49440.0,3.920510e+00,3.592779e+00,1.000000e+00,2.000000e+00,2.000000e+00,7.000000e+00,2.400000e+01
buildingclasstypeid,15.0,3.933333e+00,2.581989e-01,3.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00
architecturalstyletypeid,206.0,7.388350e+00,2.734542e+00,2.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,2.100000e+01
airconditioningtypeid,24953.0,1.813289e+00,2.967894e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.300000e+01
id,77381.0,1.495139e+06,8.609071e+05,3.490000e+02,7.520700e+05,1.497932e+06,2.240535e+06,2.982274e+06
basementsqft,50.0,6.797200e+02,6.897035e+02,3.800000e+01,2.730000e+02,5.150000e+02,7.965000e+02,3.560000e+03


**acquire & summarize**   
3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values. 

    |           |num_rows_missing |pct_rows_missing   
    | parcelid|	0|	0.000000   
    | airconditioningtypeid |	29041 |	0.535486   
    | architecturalstyletypeid |	54232 |	0.999982   


In [8]:
num_rows_missing = df.isna().sum()
num_rows_missing

parcelid                      0
typeconstructiontypeid    77159
storytypeid               77331
propertylandusetypeid         0
heatingorsystemtypeid     27941
                          ...  
buildingclassdesc         77366
heatingorsystemdesc       27941
propertylandusedesc           0
storydesc                 77331
typeconstructiondesc      77159
Length: 70, dtype: int64

In [9]:
dfmissing = pd.DataFrame(num_rows_missing, columns=['num_rows_missing'])

dfmissing.head()

,num_rows_missing
parcelid,0
typeconstructiontypeid,77159
storytypeid,77331
propertylandusetypeid,0
heatingorsystemtypeid,27941


In [10]:
dfmissing['pct_rows_missing'] = dfmissing.num_rows_missing/df.shape[0]
dfmissing.head()

,num_rows_missing,pct_rows_missing
parcelid,0,0.000000
typeconstructiontypeid,77159,0.997131
storytypeid,77331,0.999354
propertylandusetypeid,0,0.000000
heatingorsystemtypeid,27941,0.361083


In [11]:
def get_missing_rows(df):
    '''
    Write a function that takes in a dataframe of observations and attributes and returns a dataframe
    where each row is an atttribute name, the first column is the number of rows with missing values 
    for that attribute, and the second column is percent of total rows that have missing values for that 
    attribute. Run the function and document takeaways from this on how you want to handle missing values
    '''
    # find the number of rows in each column that are missing values
    num_rows_missing = df.isna().sum()
    # create new df with just that column
    dfrows = pd.DataFrame(num_rows_missing, columns=['num_rows_missing'])
    # add a calculation of % missing to the new df
    dfrows['pct_rows_missing'] = dfrows.num_rows_missing/df.shape[0]
    # return the new df
    return dfrows

In [12]:
zrows = get_missing_rows(df)
zrows.head(35)

,num_rows_missing,pct_rows_missing
parcelid,0,0.000000
typeconstructiontypeid,77159,0.997131
storytypeid,77331,0.999354
propertylandusetypeid,0,0.000000
heatingorsystemtypeid,27941,0.361083
buildingclasstypeid,77366,0.999806
architecturalstyletypeid,77175,0.997338
airconditioningtypeid,52428,0.677531
id,0,0.000000
basementsqft,77331,0.999354


In [13]:
zrows.tail(35)

,num_rows_missing,pct_rows_missing
pooltypeid7,62339,0.805611
propertycountylandusecode,0,0.000000
propertyzoningdesc,27050,0.349569
rawcensustractandblock,0,0.000000
regionidcity,1471,0.019010
regionidcounty,0,0.000000
regionidneighborhood,46491,0.600806
regionidzip,50,0.000646
roomcnt,0,0.000000
threequarterbathnbr,67285,0.869529


****
**take aways**
1. could fireplace, garage, pool, hottub, deck be made to 0 or 1 then summed as "plus_item" column?
    - this would assume null values do not have the feature, as opposed to feature is present but not noted
2. drop features with 70% or more missing values to start
****

4. Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

num_cols_missing	pct_cols_missing	num_rows   
23	38.333	108   
24	40.000	123   
25	41.667	5280   

In [14]:
# rephrase of question:
# for each observation how many columns are missing values?
# what is the % of columns with missing values for that row?
# group the rows with the same ansers to those 2 questions together

In [15]:
df.head()

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,...,logerror,pid,tdate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,14297519,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1727539,NaN,...,0.025595,14297519,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,17052889,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1387261,NaN,...,0.055619,17052889,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,14186244,NaN,NaN,261.0,NaN,NaN,NaN,NaN,11677,NaN,...,0.005383,14186244,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,12177905,NaN,NaN,261.0,2.0,NaN,NaN,NaN,2288172,NaN,...,-0.103410,12177905,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,10887214,NaN,NaN,266.0,2.0,NaN,NaN,1.0,1970746,NaN,...,0.006940,10887214,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN


In [16]:
# this will add a column that has a total number of columns that are blank for that row
df['null_count'] = df.isna().sum(axis=1)
null_count = df.null_count
df.head()

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,...,pid,tdate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc,null_count
0,14297519,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1727539,NaN,...,14297519,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN,36
1,17052889,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1387261,NaN,...,17052889,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN,33
2,14186244,NaN,NaN,261.0,NaN,NaN,NaN,NaN,11677,NaN,...,14186244,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN,34
3,12177905,NaN,NaN,261.0,2.0,NaN,NaN,NaN,2288172,NaN,...,12177905,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN,32
4,10887214,NaN,NaN,266.0,2.0,NaN,NaN,1.0,1970746,NaN,...,10887214,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN,29


In [17]:
# this calculates the percentage of null columns for that row
df['pct_null'] = df.null_count/df.shape[1]
pct_null = df.pct_null
df.head()

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,...,tdate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc,null_count,pct_null
0,14297519,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1727539,NaN,...,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN,36,0.507042
1,17052889,NaN,NaN,261.0,NaN,NaN,NaN,NaN,1387261,NaN,...,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN,33,0.464789
2,14186244,NaN,NaN,261.0,NaN,NaN,NaN,NaN,11677,NaN,...,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN,34,0.478873
3,12177905,NaN,NaN,261.0,2.0,NaN,NaN,NaN,2288172,NaN,...,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN,32,0.450704
4,10887214,NaN,NaN,266.0,2.0,NaN,NaN,1.0,1970746,NaN,...,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN,29,0.408451


In [18]:
# this gets a dataframe with just the 2 new columns
dfcol = pd.DataFrame(null_count, columns=['null_count'])
dfcol['pct_null'] = pct_null
dfcol.head()

,null_count,pct_null
0,36,0.507042
1,33,0.464789
2,34,0.478873
3,32,0.450704
4,29,0.408451


In [19]:
# this shows how many groups of rows have the same number of null columns
dfcol.nunique()

null_count    26
pct_null      26
dtype: int64

In [20]:
# create a series that has the number of rows in each group
num_rows_ingroup = dfcol.null_count.value_counts()
# create a dataframe with the count of null_count and pct_null
groups = dfcol.groupby(['null_count', 'pct_null']).count()

In [22]:
# create a df from the num_rows_ingroup, rename the columns, sort, and reset the index 
dfnum_rows = pd.DataFrame(num_rows_ingroup)
dfnum_rows = dfnum_rows.reset_index()
dfnum_rows = dfnum_rows.rename(columns={'index': 'num_null_col', 'null_count': 'num_rows_with_count'})
dfnum_rows = dfnum_rows.sort_values('num_null_col')
dfnum_rows = dfnum_rows.reset_index()

In [23]:
#visual check
dfnum_rows

,index,num_null_col,num_rows_with_count
0,25,23,2
1,21,24,13
2,19,25,24
3,15,26,65
4,11,27,316
5,10,28,455
6,5,29,5270
7,8,30,3455
8,3,31,9891
9,2,32,12579


In [24]:
# reset the index on the groups df so that we can add the num_rows_with_count
groups = groups.reset_index()

In [25]:
# visual check
groups

,null_count,pct_null
0,23,0.323944
1,24,0.338028
2,25,0.352113
3,26,0.366197
4,27,0.380282
5,28,0.394366
6,29,0.408451
7,30,0.422535
8,31,0.436620
9,32,0.450704


In [26]:
# combine num_rows_with_count from dfnum_rows with groups
groups['rows_with_count'] = dfnum_rows.num_rows_with_count

In [27]:
# visual check
groups

,null_count,pct_null,rows_with_count
0,23,0.323944,2
1,24,0.338028,13
2,25,0.352113,24
3,26,0.366197,65
4,27,0.380282,316
5,28,0.394366,455
6,29,0.408451,5270
7,30,0.422535,3455
8,31,0.436620,9891
9,32,0.450704,12579


In [29]:
# create a function that does the abocve and returns groups df
def get_missing_cols(df):
    # add calculation columns to original df
    df['null_count'] = df.isna().sum(axis=1)
    df['pct_null'] = df.null_count/df.shape[1]
    
    # create a dataframe with just the 2 new columns
    dfcol = pd.DataFrame(null_count, columns=['null_count'])
    dfcol['pct_null'] = df.pct_null
    
    # create a series that has the number of rows in each group
    num_rows_ingroup = dfcol.null_count.value_counts()
    
    # create a dataframe with the count of null_count and pct_null
    groups = dfcol.groupby(['null_count', 'pct_null']).count()
    
    # create a df from the num_rows_ingroup, rename the columns, sort, and reset the index 
    dfnum_rows = pd.DataFrame(num_rows_ingroup)
    dfnum_rows = dfnum_rows.reset_index()
    dfnum_rows = dfnum_rows.rename(columns={'index': 'num_null_col', 'null_count': 'num_rows_with_count'})
    dfnum_rows = dfnum_rows.sort_values('num_null_col')
    dfnum_rows = dfnum_rows.reset_index()
    
    # reset the index on the groups df so that we can add the num_rows_with_count
    groups = groups.reset_index()
    
    # combine num_rows_with_count from dfnum_rows with groups
    groups['rows_with_count'] = dfnum_rows.num_rows_with_count
    return groups


In [30]:
zcols = get_missing_cols(df)
zcols

,null_count,pct_null,rows_with_count
0,23,0.319444,2
1,24,0.333333,13
2,25,0.347222,24
3,26,0.361111,65
4,27,0.375000,316
5,28,0.388889,455
6,29,0.402778,5270
7,30,0.416667,3455
8,31,0.430556,9891
9,32,0.444444,12579


****
**take aways**
- most rows have 32-34 columns with missing values
****

In [ ]:
Prepare

Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

The input:
A dataframe
A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
The output:
The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.
hint:
Look up the dropna documentation.
You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.
Make use of inplace, i.e. inplace=True/False.
Decide how to handle the remaining missing values:

Fill with constant value.
Impute with mean, median, mode.
Drop row/column
wrangle_zillow.py

Functions of the work above needed to acquire and prepare a new sample of data.

Mall Customers

notebook

Acquire data from mall_customers.customers in mysql database.
Summarize data (include distributions and descriptive statistics).
Detect outliers using IQR.
Split data (train-test-split).
Encode categorical columns using a one hot encoder.
Handles missing values.
Scaling
wrangle_mall.py

Acquire data from mall_customers.customers in mysql database.
Split the data
One-hot-encoding
Missing values
Scaling